In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

%matplotlib inline

img = cv2.imread('messi5.jpg')
print(img.shape)
plt.imshow(img)

In [ ]:
# opencv 默认使用 BGR编码，相比RGB，第三个维度是反的，需要转换为 RGB

# 方法1. 直接操作数组
fig = plt.figure(figsize=(10, 6))
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)
ax1.imshow(img[:,:,np.array([2,1,0])])
# 方法2. 调用 opencv 函数
ax2.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

In [ ]:
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img_gray_small = cv2.resize(img_gray, (360, 240))
plt.imshow(img_gray_small, cmap="gray")


In [ ]:
cv2.imwrite("gray_out.png",img_gray_small)

In [ ]:
ball = img[280:340, 330:390]
img[273:333, 100:160] = ball
cv2.putText(img,"Messi",(10,50),  cv2.FONT_HERSHEY_PLAIN, 4,(255,255,255),2,cv2.LINE_AA)

plt.imshow(img[:,:,np.array([2,1,0])])

In [ ]:
img_jizhiQR = cv2.imread("./3.png")
img_jizhiQR = cv2.cvtColor(img_jizhiQR, cv2.COLOR_BGR2GRAY)
print(img_jizhiQR.shape)
plt.imshow(img_jizhiQR, "gray")

In [ ]:
# 看看取值情况，发现只有0（黑色）和 255（白色）
set(img_jizhiQR.ravel())

In [ ]:
import seaborn as sns
import tensorflow as tf
from sklearn.cluster import KMeans
sns.set_style('white')

def QR_code_to_mat(img_input, num_out):
    np_rawToQr_idx = np.linspace(0, num_out-0.1, img_input.shape[0]).astype(np.int)
    np_rawToQr_mesh = np.meshgrid(np_rawToQr_idx,np_rawToQr_idx)
    np_QR_counts = np.zeros([num_out, num_out])
    for row_idx in range(img_input.shape[0]):
        for col_idx in range(img_input.shape[1]):
            col_num_25 = np_rawToQr_mesh[0][row_idx][col_idx]
            row_num_25 = np_rawToQr_mesh[1][row_idx][col_idx]
            #print(row_idx, col_idx)
            if img_input[row_idx][col_idx] == 255:
                np_QR_counts[row_num_25][col_num_25] += 1
    
    
    return np_QR_counts

def AvgPool(img_input, num_out):
    mat_input = tf.placeholder(tf.float32)
    k_size = int(img_input.shape[0]/num_out)+1
    op = tf.nn.avg_pool(value=mat_input, ksize=[1,9,9,1], strides=[1, 9, 9, 1], padding='SAME')
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        res = (sess.run(op, feed_dict={mat_input: img_input[np.newaxis,:,:,np.newaxis]}))
    
    bigger_label = 1
    kmeans = KMeans(n_clusters=2, random_state=0).fit(res.ravel().reshape(-1,1))
    if kmeans.cluster_centers_[0] > kmeans.cluster_centers_[1]:
        bigger_label = 0
    
    threshold = min(res.ravel()[kmeans.labels_==bigger_label])
    return res[0,:,:,0] > threshold


np_25_counts = QR_code_to_mat(img_jizhiQR, 25)
np_25_counts_tf = AvgPool(img_jizhiQR, 25)

fig = plt.figure(figsize=(15,5))
ax1 = fig.add_subplot(131)
ax2 = fig.add_subplot(132)
ax3 = fig.add_subplot(133)

sns.distplot(np_25_counts.ravel(), ax=ax1)
ax2.imshow(np_25_counts>40, cmap="gray")
ax3.imshow(np_25_counts_tf, cmap="gray")


In [ ]:
img_jizhiQR = cv2.imread("./jizhi_qinding.png")
img_jizhiQR = img_jizhiQR[:,:,::-1]

# 去掉边缘的白色
img_jizhiQR_gray = cv2.cvtColor(img_jizhiQR, cv2.COLOR_RGB2GRAY)
np_totalRow = np.arange(img_jizhiQR.shape[0])
idx_rowUsed = np_totalRow[img_jizhiQR_gray.mean(0) != 255]
idx_colUsed = np_totalRow[img_jizhiQR_gray.mean(1) != 255]
img_jizhiQR_rmBlank = img_jizhiQR[idx_rowUsed,:,:][:,idx_colUsed,:]

# 去掉蓝色部分
img_jizhiQR_new = np.zeros([img_jizhiQR_rmBlank.shape[0], img_jizhiQR_rmBlank.shape[1]])
idx_black = (img_jizhiQR_rmBlank[:,:,0]<10)*(img_jizhiQR_rmBlank[:,:,1]<10)*(img_jizhiQR_rmBlank[:,:,2]<10)
img_jizhiQR_new[idx_black] = 255
np_25_counts_tf = AvgPool(img_jizhiQR_new, 29)

fig = plt.figure(figsize=(15,5))
ax1 = fig.add_subplot(131)
ax2 = fig.add_subplot(132)
ax3 = fig.add_subplot(133)

ax1.imshow(img_jizhiQR)
ax2.imshow(img_jizhiQR_new)
ax3.imshow(np_25_counts_tf)

In [ ]:
ball.shape

In [ ]:
plt.imshow(ball)

In [ ]:
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(332)
for r in ball.reshape(-1, 3):
    ax.plot(r[1], r[0], '.', c=(r[0]/255., r[1]/255., r[2]/255.) )
    
ax = fig.add_subplot(333)
for r in ball.reshape(-1, 3):
    ax.plot(r[2], r[0], '.', c=(r[0]/255., r[1]/255., r[2]/255.) )
    
ax = fig.add_subplot(336)
for r in ball.reshape(-1, 3):
    ax.plot(r[2], r[1], '.', c=(r[0]/255., r[1]/255., r[2]/255.) )
    
for i,color in enumerate(['Red', "Green", "Blue"]):
    ax = fig.add_subplot(3,3,i*3+i+1)
    ax.text(5,5, color)
    ax.plot(0,0)
    ax.plot(10,10)
    ax.set_xlim(0, 10)
    ax.set_ylim(0, 10)
    ax.axis('off')

In [ ]:
fig = plt.figure(figsize=(15, 5))
ax1 = fig.add_subplot(131)
ax2 = fig.add_subplot(132)
ax3 = fig.add_subplot(133)
ax1.imshow( (ball[:,:,0]>200) )
ax2.imshow( (ball[:,:,0]>130)+(ball[:,:,0]<50) )
ax3.imshow( (ball[:,:,0]>130)+(ball[:,:,0]<50)+(ball[:,:,1]<120) )

In [ ]:
fig = plt.figure(figsize=(12, 5))
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)

img1 = ((img[:,:,0]>130)+(img[:,:,0]<50)+(img[:,:,1]<120)).astype(np.uint8)
ax1.imshow(img)
ax2.imshow(img1)

In [ ]:
kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(3, 3))

img_e   = cv2.erode(img1, kernel, iterations=2)
img_de  = cv2.dilate(img_e, kernel, iterations=8)
img_ede = cv2.erode(img_de, kernel, iterations=3)
fig = plt.figure(figsize=(15, 5))
ax1 = fig.add_subplot(131)
ax2 = fig.add_subplot(132)
ax3 = fig.add_subplot(133)
ax1.imshow(img_e)
ax2.imshow(img_de)
ax3.imshow(img_ede)
ax1.set_title(u"first erode 2 pixels")
ax2.set_title(u"then dilate 8 pixels")
ax3.set_title(u"finally erode 3 pixels")

In [ ]:
sobelx = cv2.Sobel(img_ede*255, cv2.CV_64F, 1, 0)
sobely = cv2.Sobel(img_ede*255, cv2.CV_64F, 0, 1)
img_sob = np.sqrt(sobelx**2+sobely**2).astype(np.uint8)
plt.imshow(img_sob)

In [ ]:
gray = img_sob
canny = cv2.Canny(gray, 200, 300)
gray = cv2.medianBlur(gray, 5)
np_hc = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT, dp=1, minDist=60,
              param1=200,
              param2=10,
              minRadius=20,
              maxRadius=30)

fig = plt.figure()
ax = fig.add_subplot(111)
ax.imshow(img_sob, "gray")
img_tmp = np_hc
for i in range(np_hc.shape[1]):
    img_tmp = cv2.circle(img, (np_hc[0,i,0],np_hc[0,i,1]), np_hc[0,i,2], (255, 0, 0), 8)

plt.imshow(img_tmp)


In [ ]:
#!wget -O ./udacity_vehicle.tar.gz http://dl4img-1251985129.cosbj.myqcloud.com/udacity_vehicle.tar.gz
#!mkdir -p ./dataset
#!tar -zxvf udacity_vehicle.tar.gz -C ./dataset

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import sys
import cv2
import scipy.stats as stats
from tqdm import tqdm
import os
from sklearn.metrics import confusion_matrix, auc, roc_auc_score
import matplotlib.pyplot as plt
import sklearn
from skimage.feature import hog
import pandas as pd
import os
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC,SVC
from sklearn.preprocessing import StandardScaler
import time
from scipy.ndimage.measurements import label
import numpy as np
import functools
import pickle


l_samp = !ls ./dataset/*vehicles/*/*

M_ClassDict = {"non-vehicles" : 0, "vehicles" : 1}
pd_SampClass = pd.DataFrame({
    "Sample" : l_samp, 
    "Class"   : list(map(lambda x: M_ClassDict[x], list(map(lambda x: x.split("/")[2], l_samp))))
})[['Sample', 'Class']]
pd_SampClass_train,pd_SampClass_cv = train_test_split(pd_SampClass, test_size=0.33, random_state=42)
pd_SampClass_train.head()

In [ ]:
fig = plt.figure(figsize=(12, 6))
for i in range(5):
    image = cv2.imread(pd_SampClass_train['Sample'].iloc[i])
    image = image[:,:,::-1]
    ax = fig.add_subplot(1,5,i+1)
    ax.imshow(image)
    ax.set_title(pd_SampClass_train['Class'].iloc[i])


In [ ]:
fig = plt.figure(figsize=(20, 10))

img = cv2.imread("./dataset/vehicles/GTI_Far/image0000.png")
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

for i1,pix_per_cell in enumerate([6,8,10]):
    for i2,cell_per_block in enumerate([2,3]):
        for i3,orient in enumerate([6,8,9]):
            features, hog_image = hog(img_gray, pixels_per_cell=(pix_per_cell,pix_per_cell), 
                               cells_per_block=(cell_per_block,cell_per_block),
                               orientations=orient, visualise=True, feature_vector=False
                         )
            #print(features.shape)
            ax = fig.add_subplot(3,6,i1*6+i2*3+i3+1)
            ax.imshow(hog_image, 'gray')
            ax.set_title("Pix%d_C%d_Ori%d" % (pix_per_cell, cell_per_block, orient))

In [ ]:
# 这里只看灰度图的轮廓，不考虑颜色。如果需要考虑，这里可以继续添加
l_colorSpace = [cv2.COLOR_BGR2GRAY]
l_names = ["GRAY"]
l_len   = [1]

def get_hog_features(img, orient, pix_per_cell=8, cell_per_block=2,
                        vis=False, feature_vec=True):

    if vis == True:
        features, hog_image = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block), transform_sqrt=True,
                                  visualise=vis, feature_vector=feature_vec)
        return features, hog_image

    else:
        features = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block), transform_sqrt=True,
                       visualise=vis, feature_vector=feature_vec)
        return features

def get_features(img, pix_per_cell=8,cell_per_block=2,orient=9, getImage=False, inputFile=True, feature_vec=True):
    l_imgLayers = []
    for cs in l_colorSpace:
        if inputFile:
            l_imgLayers.append(cv2.cvtColor(cv2.imread(img), cs))
        else:
            l_imgLayers.append(cv2.cvtColor(img, cs))

    l_hog_features = []
    l_images = []
    for feature_image in l_imgLayers:
        hog_features = []
        n_channel = 1
        if len(feature_image.shape) > 2:
            n_channel = feature_image.shape[2]
        for channel in range(n_channel):
            featureImg = feature_image
            if n_channel > 2:
                featureImg = feature_image[:,:,channel]

            vout,img = get_hog_features(featureImg,
                                orient, pix_per_cell, cell_per_block,
                                vis=True, feature_vec=feature_vec)
            if getImage:
                l_images.append(img)
            #print(featureImg.shape, vout.shape)
            hog_features.append(vout)

        l_hog_features.append(list(hog_features) )

    if getImage:
        return l_images
    else:
        return functools.reduce(lambda x,y: x+y, l_hog_features)


In [ ]:
if os.path.isfile("./X_train.npy") == 0:
    l_X_train = []
    l_X_test  = []
    for r in tqdm(pd_SampClass_train.iterrows()):
        l_X_train.append(np.array(get_features(r[1]['Sample'])).ravel())

    for r in tqdm(pd_SampClass_cv.iterrows()):
        l_X_test.append( np.array(get_features(r[1]['Sample'])).ravel())
    
    X_train = np.array(l_X_train)
    X_test  = np.array(l_X_test)
    np.save("./X_train.npy", X_train)
    np.save("./X_test.npy",  X_test)
else:
    X_train = np.load("./X_train.npy")
    X_test  = np.load("./X_test.npy")

y_train = pd_SampClass_train['Class'].values
y_test  = pd_SampClass_cv['Class'].values

In [ ]:
X_scalerM = StandardScaler()
X_trainT = X_scalerM.fit_transform(X_train)
X_testT  = X_scalerM.transform(X_test)

X_trainTs,y_trainTs = sklearn.utils.shuffle(X_trainT, y_train)

In [ ]:
svc = SVC(random_state=0, C=1)
t=time.time()
svc.fit(X_trainTs, y_trainTs)
t2 = time.time()
print(round(t2-t, 2), 'Seconds to train SVC...')

In [ ]:
print('Test Accuracy of SVC = ', round(svc.score(X_testT, y_test), 4))
t=time.time()
n_predict = 10
print('My SVC predicts: ', svc.predict(X_testT[0:n_predict]))
print('For these',n_predict, 'labels: ', y_test[0:n_predict])
t2 = time.time()

pred = svc.predict(X_testT)
print("AUC for Merge dataset = %1.2f,\n" % (roc_auc_score(pred, y_test)))
print(confusion_matrix(pred, y_test))